# Part 1.2 - Extract Bottleneck Features From RNN 
In this notebook, we will load a pre-trained RNN model and run both our train & test timeseries samples through it, extracting the bottleneck features (the outputs of the layer just before the fully-connected / classification layer(s)).

The bottleneck features will be used to represent the embedded timeseries features for each sample. In the next notebook, we will concatenate them with other features to train our final XGBoost classification model. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import math
import pandas as pd
import numpy as np
import time
import tensorflow as tf
from rnn import PlasticcRNN
import matplotlib.pyplot as pltß
%matplotlib inline

print(tf.__version__)

### Load Train & Test Data

In [ ]:
train = pd.read_pickle('train_rnn.pkl')
test = pd.read_pickle('test_rnn.pkl')

### Load pre-trained RNN model

You can find the code for the RNN model in `rnn.py` if you'd like to take a look at the implementation.

In [ ]:
model = PlasticcRNN('weight/rnn.npy')

We use the function `predict_bottleneck()` to feed each training example through the pre-trained RNN model and extract the outputs from the layer just before the final classification layer (the bottleneck layer).

In [ ]:
train_bn = model.predict_bottleneck(train)

We also extract the bottleneck features for the test dataset.

In [ ]:
test_bn = model.predict_bottleneck(test)

Let's verify that we have embedded each of our timeseries into the 16-dimensional space. 

In [ ]:
print(train_bn.shape)
print(test_bn.shape)

### Convert Bottleneck Features to DataFrames

Since we will be concatenating these bottleneck features with a Dataframe in the next notebook, we can create dataframes for our bottleneck features.

In [ ]:
train_bn = pd.DataFrame(train_bn,columns=['bottleneck%d'%i for i in range(train_bn.shape[1])])
train_bn['object_id'] = train.object_id.unique().astype("int32")

In [ ]:
train_bn.head()

In [ ]:
test_bn = pd.DataFrame(test_bn,columns=['bottleneck%d'%i for i in range(test_bn.shape[1])])
test_bn['object_id'] = test.object_id.unique().astype("int32")

### Store Features to Disk

In [ ]:
train_bn.to_pickle('train_bn.pkl')
test_bn.to_pickle('test_bn.pkl')